In [1]:
import yaml
import requests
# Draw Map of indonesia using folium and pin the locations of the addresses
import folium
import geocoder


In [2]:
with open("certs/certs.yaml", "r") as f:
    config = yaml.safe_load(f)
    api_key = config["google_api_key"]

In [3]:
# export address to yaml
with open("address.yaml", "r") as f:
    adrs = yaml.safe_load(f)    
    address_urls = adrs["addresses"]

In [4]:
address_urls

[{'red': ['https://maps.app.goo.gl/FjwqoJMeScaRAyEb7',
   'https://maps.app.goo.gl/Qg8bkEak6P5npsan9',
   'https://maps.app.goo.gl/gUGFkHFh9FmVHNbB6']},
 {'blue': ['Kampus ITB Bandung']},
 {'green': ['Telkom Corporate University']}]

In [17]:
def get_address_from_gmap_url(url: str) -> str:
    if "http" in url:
        # Send a GET request and allow redirection
        response = requests.get(url, allow_redirects=True)

        # Get the final URL after redirection
        final_url = response.url

        # Split the URL to get the location part
        location_part = final_url.split('/place/')[1].split('/')[0]

        # Replace '+' with spaces to format the location name
        location_name = location_part.replace('+', ' ')

        # Print the location name
        return location_name
    else:
        return url

def get_longlat_from_address(address:str, api_key:str) -> dict:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    long = data["results"][0]["geometry"]["location"]["lng"]
    lat = data["results"][0]["geometry"]["location"]["lat"]

    return {"address": address, "longitude":long, "latitude":lat}

In [15]:
address_urls

[{'red': ['https://maps.app.goo.gl/FjwqoJMeScaRAyEb7',
   'https://maps.app.goo.gl/Qg8bkEak6P5npsan9',
   'https://maps.app.goo.gl/gUGFkHFh9FmVHNbB6']},
 {'blue': ['Kampus ITB Bandung']},
 {'green': ['Telkom Corporate University']}]

In [21]:

full_addresses = []
for address in address_urls:
    for urlss in address[list(address.keys())[0]]:
        addr = get_address_from_gmap_url(urlss)
        full_address = get_longlat_from_address(addr, api_key)
        full_address.update({"color": f"{list(address.keys())[0]}"})
        full_addresses.append(full_address)
        print(address)

{'red': ['https://maps.app.goo.gl/FjwqoJMeScaRAyEb7', 'https://maps.app.goo.gl/Qg8bkEak6P5npsan9', 'https://maps.app.goo.gl/gUGFkHFh9FmVHNbB6']}
{'red': ['https://maps.app.goo.gl/FjwqoJMeScaRAyEb7', 'https://maps.app.goo.gl/Qg8bkEak6P5npsan9', 'https://maps.app.goo.gl/gUGFkHFh9FmVHNbB6']}
{'red': ['https://maps.app.goo.gl/FjwqoJMeScaRAyEb7', 'https://maps.app.goo.gl/Qg8bkEak6P5npsan9', 'https://maps.app.goo.gl/gUGFkHFh9FmVHNbB6']}
{'blue': ['Kampus ITB Bandung']}
{'green': ['Telkom Corporate University']}


In [22]:
full_addresses

[{'address': "62RM%2BW86 Mujagi Farmers' Group, Cipendawa, Pacet, Cianjur Regency, West Java 43253",
  'longitude': 107.0332728,
  'latitude': -6.7577098,
  'color': 'red'},
 {'address': '5HV2%2B3PM Sinar Mukti Kelompok Tani, Tugu, Tugumukti, Kec. Cisarua, Kabupaten Bandung Barat, Jawa Barat 40551',
  'longitude': 107.5571052,
  'latitude': -6.8081473,
  'color': 'red'},
 {'address': 'Hikmah Farm Benih Bibit Kentang Bersertifikat, Jl. PTPN VIII Kertamanah No.1, Pangalengan, Kec. Pangalengan, Kabupaten Bandung, Jawa Barat 40378',
  'longitude': 107.5858717,
  'latitude': -7.1842683,
  'color': 'red'},
 {'address': 'Kampus ITB Bandung',
  'longitude': 107.6104924,
  'latitude': -6.8912926,
  'color': 'blue'},
 {'address': 'Telkom Corporate University',
  'longitude': 107.5890164,
  'latitude': -6.8712049,
  'color': 'green'}]

In [25]:
import json
geo_data= "indonesia-geojson/indonesia-province.json" 
with open(geo_data) as f:
    geo_data = json.load(f)

In [26]:
without_jabar = [x for x in geo_data['features'] if x['properties']['Propinsi'] != 'JAWA BARAT']
geo_data['features'] = without_jabar

In [29]:
def plot_folium_map(addresses: list, geojson: str = None, zoom_start: int = 8.5, pin:str = "pin"):
    m = folium.Map(location=[-6.890910999999999, 107.668887], 
               zoom_start=zoom_start, 
               control_scale=False,
               zoom_control=False,
               attr="Mapbox Bright"
               )

    if geojson is not None:
        geo_data= geojson
        folium.GeoJson(geo_data).add_to(m)
        # white all geojson color opaque 100%, border white 
        style_function = lambda x: {'fillColor': '#ffffff', 'color': 'white', 'weight': 2, 'fillOpacity': 1}
        folium.GeoJson(geo_data, style_function=style_function).add_to(m)

    for address in addresses:
        if pin == "circle":
            folium.CircleMarker([address["latitude"], address["longitude"]], radius=5, color=address["color"], fill=True, fill_color=address["color"], fill_opacity=1).add_to(m)
        if pin == "pin":
            folium.Marker([address["latitude"], address["longitude"]], popup=address["address"]).add_to(m)
    
    return m

In [30]:

m = plot_folium_map(full_addresses, geojson=geo_data, zoom_start=8.5, pin= 'circle')
m.save('map.html')